In [90]:
import numpy as np
import cv2
import autopy
import logging
import cmake
import face_recognition

#importação das bibliotecas

ModuleNotFoundError: No module named 'face_recognition'

In [91]:
import pytesseract as ocr
import numpy as np
import cv2


from PIL import Image

imagem = Image.open('livro1  .jpg').convert('RGB')

npimagem =np.asarray(imagem).astype(np.uint8)

npimagem[:, :, 0] = 0
npimagem[:, :, 2] = 0

im = cv2.cvtColor(npimagem, cv2.COLOR_RGB2GRAY) 


ret, thresh = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

binimagem = Image.fromarray(thresh)

phrase = ocr.image_to_string(binimagem, lang = 'por')

print(phrase)

ModuleNotFoundError: No module named 'pytesseract'

In [89]:
faces_encodings = []
faces_names = []
cur_direc = os.getcwd()
path = os.path.join(cur_direc, 'faces/')
list_of_files = [f for f in glob.glob(path+'*.jpg')]
number_files = len(list_of_files)
names = list_of_files.copy()

NameError: name 'os' is not defined

In [85]:
log_format = '%(asctime)s:%(levelname)s:%(filename)s:%(message)s'

logging.basicConfig(filename='trabalho-comgraf-g2.log',
                    # w -> sobrescreve o arquivo a cada log
                    # a -> não sobrescreve o arquivo
                    filemode='w',
                    level=logging.DEBUG,
                    format=log_format)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [86]:
def fullname(primeiro_nome: str, segundo_nome: str) -> str:

    # Aqui, verificamos se os parametros passados são do tipo string (str)
    if isinstance(primeiro_nome, str) and isinstance(segundo_nome, str):
        logger.info(f'{primeiro_nome} {segundo_nome}')
        return primeiro_nome + segundo_nome
    else:
        logger.error(
            f'{primeiro_nome} type: {type(primeiro_nome)} - {segundo_nome} type: {type(segundo_nome)}'
        )

fullname('Manu', 'Marques')
fullname('Manu', True)

logging.info('foi quando o user logou')

In [59]:
wCam, hCam = 640, 480 #define tamanho da camera
frameR = 100     #Redução do frame
smoothening = 7  #usado pra adequar o movimento do mouse - valor mágico

pTime = 0
plocX, plocY = 0, 0
clocX, clocY = 0, 0
wScr, hScr = autopy.screen.size() #retorna o tamanho da tela

In [60]:
def prep(captured_frame):
    captured_frame_bgr = cv2.cvtColor(captured_frame, cv2.COLOR_BGRA2BGR) #remove a opacidade
    captured_frame_bgr = cv2.medianBlur(captured_frame_bgr, 3) #deixa a imagem com blur
    captured_frame_lab = cv2.cvtColor(captured_frame_bgr, cv2.COLOR_BGR2Lab) #converte a cor pra cor lab
    
    return captured_frame_lab

In [61]:
def filter_by(captured_frame,init, end):
    filter_range = cv2.inRange(captured_frame, np.array(init), np.array(end)) #cria uma máscara baseada no range init e end
    img_filter = cv2.GaussianBlur(filter_range, (5, 5), 2, 2) #converte a máscara pra uma diferença de gaussian
    
    return img_filter

In [62]:
def get_contours(img):
    canny = cv2.Canny(img,10,50) #converte a imagem para retornar bordas -pega um objeto sólido e faz o contorno - o interior fica preto
    contours = cv2.findContours(canny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)[0] #pega as coordenadas das bordas
    
    return contours

In [63]:
def add_retangle(img, contour, color):
    peri = cv2.arcLength(cnt, True) #calcula o perimetro       
    approx = cv2.approxPolyDP(cnt, 0.02* peri,True) #retorna as coordenadas aproximadas
    cv2.drawContours(img, cnt, -1, (255, 0, 255), 7) #desenha as bordas
    x_,y_, w, h = cv2.boundingRect(approx) #retorna as coordenadas quadraticas
    cv2.rectangle(img, (x_,y_), (x_+w,y_+h),color, 5) #faz o quadrado baseado nas coordenadas

In [64]:
cap = cv2.VideoCapture(0) #captura a camera
close = False

while(True):
    ret, captured_frame = cap.read() #pega o frame da camera
    output_frame = captured_frame.copy() #faz a copia do frame

    img_prep = prep(captured_frame) #prepara a imagem
    
    img_filter_left_click = filter_by(img_prep, [17, 63, 84], [255, 112, 255]) #objeto cor verde - filtro baseado na cor
    contours_left_click = get_contours(img_filter_left_click) # pega as bordas a partir do filtro
    
    img_filter_right_click = filter_by(img_prep, [43, 144, 90], [199, 199, 164])#objeto cor rosa - filtro baseado na cor
    contours_right_click = get_contours(img_filter_right_click) # pega as bordas a partir do filtro
    
    img_filter_close_click = filter_by(img_prep, [23, 40, 2], [179, 255, 255])#objeto cor azul - filtro baseado na cor
    contours_close_click = get_contours(img_filter_close_click) # pega as bordas a partir do filtro
    
    img_filter_move = filter_by(img_prep, [101, 69, 149], [236, 179, 190]) #objeto cor amarelo - filtro baseado na cor
    contours_move = get_contours(img_filter_move) # pega as bordas a partir do filtro
    
    for cnt in contours_left_click: #pra cada borda do filtro, executa o clique
        area = cv2.contourArea(cnt) #pega a area total da borda
        if area> 1000: #se for maior que 1000
            add_retangle(output_frame, cnt, (255, 0, 0)) #adiciona um retangulo vermelho
            autopy.mouse.click(autopy.mouse.Button.LEFT) #clica com o botao esquerdo          
        
    for cnt in contours_right_click:
        area = cv2.contourArea(cnt)
        if area> 1000:
            add_retangle(output_frame, cnt, (252, 223, 3)) #adiciona um retangulo amarelo          
            autopy.mouse.click(autopy.mouse.Button.RIGHT)              
        
    for cnt in contours_close_click:
        area = cv2.contourArea(cnt)
        if area> 1000:            
            add_retangle(output_frame, cnt, (3, 36, 252)) #adiciona um retangulo azul
            close = True
    
    for cnt in contours_move:
        area = cv2.contourArea(cnt)
        
        if area> 109:
            add_retangle(output_frame, cnt, (0,255,0)) #adiciona um retangulo verde
            
            peri = cv2.arcLength(cnt, True)  # pega o perimetro      
            approx = cv2.approxPolyDP(cnt, 0.02* peri,True) # aproximaçao - mesma regra la de cima
            
            x_,y_, w, h = cv2.boundingRect(approx)
            x1, y1 = (int(x_+w/2) , int(y_+h/2))  #pega a coordenada do centro do quadrado 
            
            x3 = np.interp(x1, (frameR, wCam-frameR), (0, wScr)) #pega a intersecção do eixo x baseada na largura da camera com a largura da tela
            y3 = np.interp(y1, (frameR, hCam-frameR), (0, hScr)) #pega a intersecção do eixo y baseada na altura da camera com a altura da tela

            # Calcula o movimento que vai fazer
            clocX = plocX + (x3 - plocX) / smoothening
            clocY = plocY + (y3 - plocY) / smoothening

            # Movimento do mouse
            autopy.mouse.move(wScr - clocX, clocY)
            cv2.circle(output_frame, (x1, y1), 15, (255, 0, 255), cv2.FILLED) #insere um circulo na coordenada que ta usando
            plocX, plocY = clocX, clocY # salva as infos
       

    cv2.imshow('frame', output_frame) #retorna a camera com as infos
    if cv2.waitKey(1) & 0xFF == ord('q'): #se quiser fechar com q
        break
        
cap.release() #liberar câmera
cv2.destroyAllWindows() #destroi todas as janelas que o cv2 abriu

In [55]:
cap.release() #liberar câmera
cv2.destroyAllWindows() #destroi todas as janelas que o cv2 abriu